In [287]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import OneHotEncoder

In [268]:
df = pd.read_csv('data/Churn.csv')

In [269]:
df.shape

(7043, 21)

In [270]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [271]:
df['TotalCharges'] = df['TotalCharges'].replace(' ', 0)
df['TotalCharges'] = df['TotalCharges'].astype(float)


In [272]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [273]:
df.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [274]:
df['churn'] = df['churn'].map({"No": 0, "Yes": 1})

## Split

In [275]:
df = df.drop(columns=['customerid'])

In [276]:
X = df.drop('churn', axis=1).copy()
y = df['churn'].copy()

In [277]:
x_train, x_rest, y_train, y_rest = train_test_split(X, y, train_size=0.6, random_state=42, stratify=y)
x_valid, x_test, y_valid, y_test = train_test_split(x_rest, y_rest, test_size=0.5, random_state=42, stratify=y_rest)

print("Tamaño del conjunto de entrenamiento:", x_train.shape)
print("Tamaño del conjunto de validación:", x_valid.shape)
print("Tamaño del conjunto de prueba:", x_test.shape)

Tamaño del conjunto de entrenamiento: (4225, 19)
Tamaño del conjunto de validación: (1409, 19)
Tamaño del conjunto de prueba: (1409, 19)


## EDA

`normalize=True` me muestra los porcentajes en lugar de cuantos valores hay

In [278]:
df.churn.value_counts()

churn
0    5174
1    1869
Name: count, dtype: int64

In [279]:
df.churn.value_counts(normalize=True)

churn
0    0.73463
1    0.26537
Name: proportion, dtype: float64

## Feature importance: mutual information

In [280]:
cat_columns = df.select_dtypes(include=['object']).columns
num_cols = x_train.select_dtypes(include=['number']).columns
cat_columns

Index(['gender', 'partner', 'dependents', 'phoneservice', 'multiplelines',
       'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection',
       'techsupport', 'streamingtv', 'streamingmovies', 'contract',
       'paperlessbilling', 'paymentmethod'],
      dtype='object')

In [281]:
# No me funcionaba cuando usaba una variable con altisima cardinaliad, era la del id
def calculate_mi(series):
    return mutual_info_score(series, df.churn)

mi = df[cat_columns].apply(calculate_mi)
mi.sort_values(ascending=False)

contract            0.098453
onlinesecurity      0.064677
techsupport         0.063021
internetservice     0.055574
onlinebackup        0.046792
paymentmethod       0.044519
deviceprotection    0.043917
streamingmovies     0.032001
streamingtv         0.031908
paperlessbilling    0.019194
dependents          0.014467
partner             0.011454
multiplelines       0.000801
phoneservice        0.000072
gender              0.000037
dtype: float64

## Feature importance: Correlation

In [282]:
df[num_cols].corrwith(df.churn)

seniorcitizen     0.150889
tenure           -0.352229
monthlycharges    0.193356
totalcharges     -0.198324
dtype: float64

Como churn está en 0 y 1, al calcular el ``mean()`` me ayuda a ver la relación que tienen, en este caso indica que el 58% de las personas con 2 meses o menos de permanencia se dieron de baja.

In [283]:
df[df.tenure <= 2].churn.mean()

np.float64(0.5835266821345708)

In [284]:
df[df.tenure > 2].churn.mean()

np.float64(0.22099983821388125)

## One hot encoding

In [300]:
cat_columns = df.select_dtypes(exclude=['number']).columns

onehot = OneHotEncoder(sparse_output=False)
df_encoding = onehot.fit_transform(df[cat_columns])
df_encoding = pd.DataFrame(df_encoding, columns=onehot.get_feature_names_out(cat_columns))
new_df = pd.concat([df, df_encoding], axis=1).drop(columns=cat_columns, axis=1)

new_df.columns

Index(['seniorcitizen', 'tenure', 'monthlycharges', 'totalcharges', 'churn',
       'gender_Female', 'gender_Male', 'partner_No', 'partner_Yes',
       'dependents_No', 'dependents_Yes', 'phoneservice_No',
       'phoneservice_Yes', 'multiplelines_No',
       'multiplelines_No phone service', 'multiplelines_Yes',
       'internetservice_DSL', 'internetservice_Fiber optic',
       'internetservice_No', 'onlinesecurity_No',
       'onlinesecurity_No internet service', 'onlinesecurity_Yes',
       'onlinebackup_No', 'onlinebackup_No internet service',
       'onlinebackup_Yes', 'deviceprotection_No',
       'deviceprotection_No internet service', 'deviceprotection_Yes',
       'techsupport_No', 'techsupport_No internet service', 'techsupport_Yes',
       'streamingtv_No', 'streamingtv_No internet service', 'streamingtv_Yes',
       'streamingmovies_No', 'streamingmovies_No internet service',
       'streamingmovies_Yes', 'contract_Month-to-month', 'contract_One year',
       'contract_Two

toca ahora si hacer el split, y deberia pasar los de yes y no a 1 0 